<a href="https://colab.research.google.com/github/KyPython/AI-Math-Olympiad/blob/main/AI_Math_Olympiad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Requirements

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
import google.auth

# Use google.auth.default() to get credentials
credentials, _ = google.auth.default()

drive_service = build('drive', 'v3', credentials=credentials)

import io
from googleapiclient.http import MediaIoBaseDownload

file_id = '1qZedRAnbMPBFaaws3L7xrQdG3ioF-h5uZrVbpdA-YF4' # Replace with your file ID
request = drive_service.files().export_media(fileId=file_id, mimeType='text/plain')
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

with open('/content/requirements.txt', 'wb') as f:
    f.write(fh.getbuffer())

!pip install --target=/content/drive/MyDrive/AI_Math_Olympiad/ -r /content/requirements.txt

import sys
sys.path.append('/content/drive/MyDrive/AI_Math_Olympiad/')

!pip install datasets==2.12.0
from datasets import Dataset

#!rm -rf /content/drive
#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Imports

In [ ]:
import pandas as pd
import numpy as np
import re
import logging
import torch
import torch.nn as nn
!pip install pymupdf
import fitz
import sympy
import matplotlib.pyplot as plt
import networkx as nx
import joblib
import shap
!pip install pypdf
import pypdf
import re
!pip install PyPDF2 # Install PyPDF2 module
from PyPDF2 import PdfReader  # Import PdfReader from PyPDF2

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, learning_curve
from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset, DataLoader
from datasets import Dataset

import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from datasets import Dataset as HFDataset #import Dataset from datasets library as HFDataset
import fitz
import sympy
import re
import numpy as np
import torch.nn as nn
import logging
from torch.utils.data import DataLoader
import networkx as nx
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.6 MB/s eta 0:00:00


# ***Dask Implementation If Needed***

In [ ]:
import dask.dataframe as dd
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# Example: Load a large CSV file with Dask
df = dd.read_csv("large_dataset.csv")

# Example: Perform data preprocessing with Dask
def preprocess_row(row):
    # Perform your preprocessing operations here
    # Example: Calculate a new column
    row['new_column'] = row['column1'] + row['column2']
    return row

df = df.map_partitions(lambda pdf: pdf.apply(preprocess_row, axis=1))

# Example: Convert Dask DataFrame to PyTorch Dataset
class DaskDataset(Dataset):
    def __init__(self, ddf):
        self.ddf = ddf.compute() #Convert dask dataframe to pandas dataframe.

    def __len__(self):
        return len(self.ddf)

    def __getitem__(self, idx):
        row = self.ddf.iloc[idx]
        # Extract features and labels from the row
        features = torch.tensor(row['feature1'], dtype=torch.float32)
        label = torch.tensor(row['label'], dtype=torch.long)
        return features, label

# Create PyTorch DataLoader
dataset = DaskDataset(df)
dataloader = DataLoader(dataset, batch_size=32)

# Iterate over the DataLoader for training
for features, labels in dataloader:
    # Perform model training
    # ...
    pass

Functions Defined

In [ ]:
def extract_problems_from_pdf(pdf_path):
    problems = []
    answers = []
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            try:
                text += page.extract_text() or ""
            except Exception as e:
                print(f"Error extracting text from a page: {e}")

        # Adjust the pattern as needed for your PDF's format
        problem_answer_pairs = re.findall(r"Problem\s*\d+\s*(.*?)\s*Answer:\s*(.*?)(?=\s*Problem\s*|\Z)", text, re.DOTALL)
        for problem, answer in problem_answer_pairs:
            problems.append(problem.strip())
            answers.append(answer.strip())
    except FileNotFoundError:
        print(f"PDF file not found at {pdf_path}.")
    except Exception as e:
        print(f"Error reading PDF: {e}")
    return problems, answers

def parse_latex(latex_str):
    """Parses AIMO-specific LaTeX mathematical expressions."""
    # ... (LaTeX parsing logic) ...
    try:
        latex_str = re.sub(r'\s+', ' ', latex_str).strip()
        latex_str = latex_str.replace(r'\lvert', '|')
        latex_str = latex_str.replace(r'\rvert', '|')
        latex_str = latex_str.replace(r'\left|', '|')
        latex_str = latex_str.replace(r'\right|', '|')

        latex_str = re.sub(r'\\binom{(\S+)}{(\S+)}', r'binomial(\1, \2)', latex_str)
        latex_str = re.sub(r'\\lfloor{(\S+)}\\rfloor', r'floor(\1)', latex_str)
        latex_str = re.sub(r'\\lceil{(\S+)}\\rceil', r'ceiling(\1)', latex_str)
        latex_str = re.sub(r'\{(\S+)\}', r'fractional_part(\1)', latex_str)
        latex_str = latex_str.replace(r'\mathbb{N}', 'PositiveIntegers()')
        latex_str = latex_str.replace(r'\mathbb{Z}', 'Integers()')
        latex_str = latex_str.replace(r'\mathbb{Q}', 'Rationals()')
        latex_str = latex_str.replace(r'\mathbb{R}', 'Reals()')

        def handle_overline(match):
            digits = match.group(1)
            return str(int(digits))
        latex_str = re.sub(r'\\overline\{(\d+)\}', handle_overline, latex_str)

        expr = sympy.parsing.latex.parse_latex(latex_str)

        def binomial(n, k):
            return sympy.binomial(n, k)
        def floor(x):
            return sympy.floor(x)
        def ceiling(x):
            return sympy.ceiling(x)
        def fractional_part(x):
            return sympy.frac(x)

        local_dict = {'binomial': binomial, 'floor': floor, 'ceiling': ceiling, 'fractional_part': fractional_part}

        expr = sympy.parsing.latex.parse_latex(latex_str, local_dict=local_dict)
        return expr

    except Exception as e:
        logging.error(f"LaTeX parsing error: {e}, LaTeX: {latex_str}")
        return None

def sympy_to_sequence(expr):
    """Converts a sympy expression to a sequence of tokens."""
    tokens = []
    def tokenize(expr):
        if isinstance(expr, sympy.Symbol) or isinstance(expr, sympy.Number):
            tokens.append(str(expr))
        elif isinstance(expr, sympy.Expr):
            tokens.append(str(expr.func))
            for arg in expr.args:
                tokenize(arg)
    tokenize(expr)
    return tokens

class MathDataset(Dataset):
    def __init__(self, problems, answers, tokenizer, max_length=128):
        self.problems = problems
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.problems)

    def __getitem__(self, idx):
        problem = str(self.problems[idx])  # Ensure problem is a string
        answer = str(self.answers[idx])    # Ensure answer is a string

        input_text = "solve: " + problem
        target_text = answer

        inputs = self.tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_length)
        targets = self.tokenizer(target_text, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_length)

        return {
            "input_ids": inputs.input_ids.flatten(),
            "attention_mask": inputs.attention_mask.flatten(),
            "labels": targets.input_ids.flatten()
        }

def calculate_modulo_1000_high_precision(expr):
    """Calculates modulo 1000 with high precision."""
    try:
        if isinstance(expr, sympy.Expr):
            numerical_result = expr.evalf()
        else:
            numerical_result = expr
        if isinstance(numerical_result, sympy.core.numbers.Integer):
            result_int = int(numerical_result)
        elif isinstance(numerical_result, sympy.core.numbers.Float):
            decimal_result = decimal.Decimal(str(numerical_result))
            result_int = int(decimal_result.to_integral_value(rounding=decimal.ROUND_HALF_UP))
        else:
            result_int = int(numerical_result)
        return result_int % 1000
    except Exception as e:
        print(f"Error calculating modulo 1000: {e}")
        return None

def modulo_1000_loss(outputs, labels):
    """Categorical cross-entropy loss for modulo 1000."""
    criterion = nn.CrossEntropyLoss()
    labels = labels.long()
    loss = criterion(outputs, labels)
    return loss

def evaluate_model_kfold(df, model_name="t5-base", k_folds=5, epochs=3):
    """Evaluates the model using k-fold cross-validation."""
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_results = []
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    for fold, (train_index, val_index) in enumerate(kf.split(df)):
        logging.info(f"Fold {fold + 1}/{k_folds}")
        train_df = df.iloc[train_index]
        val_df = df.iloc[val_index]
        model = T5ForConditionalGeneration.from_pretrained(model_name)
        train_dataset = MathDataset(train_df["problem"].tolist(), train_df["answer"].astype(str).tolist(), tokenizer)
        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
        for epoch in range(epochs):
            for batch in train_loader:
                optimizer.zero_grad()
                outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
                loss = outputs.loss
                loss.backward()
                optimizer.step()
            logging.info(f"Fold {fold + 1}, Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
        correct_predictions = 0
        total_predictions = 0
        for index, row in val_df.iterrows():
            latex_problem = row["problem"]
            true_answer = int(row["answer"])
            predicted_answer = predict(latex_problem, model, tokenizer)
            if predicted_answer == true_answer:
                correct_predictions += 1
            total_predictions += 1
        accuracy = correct_predictions / total_predictions
        fold_results.append(accuracy)
        logging.info(f"Fold {fold + 1} Accuracy: {accuracy}")

    # Aggregate results
    average_accuracy = np.mean(fold_results)
    std_accuracy = np.std(fold_results)
    logging.info(f"Average Accuracy: {average_accuracy}")
    logging.info(f"Standard Deviation of Accuracy: {std_accuracy}")
    return average_accuracy, std_accuracy

# Supervised Learning Training
def train_supervised_model(train_df, val_df, model_name="t5-base"):  # Change to t5-base
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

    def tokenize_data(examples):
        inputs = tokenizer(
            examples["problem"], truncation=True, padding="max_length", max_length=256
        )  # increase max_length
        labels = tokenizer(
            examples["answer"], truncation=True, padding="max_length", max_length=64
        )  # increase max_length
        return {
            "input_ids": inputs.input_ids,
            "attention_mask": inputs.attention_mask,
            "labels": labels.input_ids,
        }

    # Change Dataset to HFDataset
    train_dataset = HFDataset.from_pandas(train_df)
    train_dataset = train_dataset.map(tokenize_data, batched=True)

    # Change Dataset to HFDataset
    val_dataset = HFDataset.from_pandas(val_df)
    val_dataset = val_dataset.map(tokenize_data, batched=True)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=200,
        per_device_train_batch_size=8,
        logging_dir="./logs",
        run_name="my_t5_training_run",
        report_to="tensorboard",
        logging_steps=10,
        evaluation_strategy="epoch",
        learning_rate=1e-4,  # adjust learning rate.
        weight_decay=0.01,  # add weight decay.
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()
    return model, tokenizer

def grid_search_hyperparameters(df, model_name="t5-small"):
    """Performs GridSearchCV for hyperparameter tuning."""
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Define hyperparameter grid
    param_grid = {
        'learning_rate': [1e-4, 5e-5, 2e-5],
        'batch_size': [8, 16, 32],
        'epochs': [2,3,4]
    }

    class T5Wrapper:
        def __init__(self, learning_rate=1e-4, batch_size=8, epochs = 3):
          self.learning_rate = learning_rate
          self.batch_size = batch_size
          self.epochs = epochs
          self.model = None
          self.tokenizer = None

        def fit(self, X, y):
          self.tokenizer = T5Tokenizer.from_pretrained(model_name)
          self.model = T5ForConditionalGeneration.from_pretrained(model_name)
          train_dataset = MathDataset(X["problem"].tolist(), y.astype(str).tolist(), self.tokenizer)
          train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
          optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)

          for epoch in range(self.epochs):
              for batch in train_loader:
                  optimizer.zero_grad()
                  outputs = self.model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
                  loss = outputs.loss
                  loss.backward()
                  optimizer.step()
              logging.info(f"Epoch {epoch+1}/{self.epochs}, Loss: {loss.item()}")
          return self

        def predict(self, X):
          predictions = []
          for latex_problem in X["problem"].tolist():
            predictions.append(predict(latex_problem, self.model, self.tokenizer))
          return predictions

    # Initialize GridSearchCV
    grid_search = GridSearchCV(
        estimator=T5Wrapper(),
        param_grid=param_grid,
        cv=KFold(n_splits=3),  # Define cross-validation strategy
        scoring='accuracy',  # Define evaluation metric
        verbose=2,
        n_jobs=-1 #use all available cores.
    )

    # Fit GridSearchCV
    grid_search.fit(df, df["answer"])

    # Access best hyperparameters
    print("Best Parameters:", grid_search.best_params_)
    print("Best Score:", grid_search.best_score_)

    # Access best model
    best_model = grid_search.best_estimator_

    return best_model

def analyze_errors_by_category(df, model, tokenizer):
    """Analyzes errors based on mathematical problem categories."""

    # Example: Keyword-based categorization (customize as needed)
    def categorize_problem(latex_str):
        if re.search(r'triangle|circle|square|angle', latex_str, re.IGNORECASE):
            return "Geometry"
        elif re.search(r'x|y|z|\+|\-|/', latex_str, re.IGNORECASE):
            return "Algebra"
        elif re.search(r'\\sin|\\cos|\\tan', latex_str, re.IGNORECASE):
            return "Trigonometry"
        elif re.search(r'\\binom', latex_str, re.IGNORECASE):
            return "Combinatorics"
        else:
            return "Other"

    df["category"] = df["problem"].apply(categorize_problem)

    categories = df["category"].unique()
    error_analysis = {}

    for category in categories:
        category_df = df[df["category"] == category]
        correct_predictions = 0
        total_predictions = 0

        for index, row in category_df.iterrows():
            latex_problem = row["problem"]
            true_answer = int(row["answer"])
            predicted_answer = predict(latex_problem, model, tokenizer)
            if predicted_answer == true_answer:
                correct_predictions += 1
            total_predictions += 1

        if total_predictions > 0:
            accuracy = correct_predictions / total_predictions
        else:
            accuracy = 0.0

        error_analysis[category] = {"accuracy": accuracy, "total": total_predictions}
        logging.info(f"Category: {category}, Accuracy: {accuracy}, Total: {total_predictions}")

    return error_analysis

def analyze_modulo_1000_errors(df):
    """Analyzes patterns in incorrect modulo 1000 results."""
    error_df = df[df["predicted_answer"] != df["true_answer"]]
    error_df["difference"] = error_df["predicted_answer"] - error_df["true_answer"]

    # Difference Analysis
    print("Error Difference Statistics:")
    print(error_df["difference"].describe())

    # Digit Analysis (Example: Last digit analysis)
    error_df["predicted_last_digit"] = error_df["predicted_answer"].apply(lambda x: x % 10)
    print("\nPredicted Last Digit Counts:")
    print(error_df["predicted_last_digit"].value_counts())

    # LaTeX Pattern Matching (Example: Fractions)
    fraction_errors = error_df[error_df["problem"].str.contains(r"\\frac")]
    print("\nFraction Errors:")
    print(fraction_errors.head())

    # Example: error histogram.
    import matplotlib.pyplot as plt
    plt.hist(error_df["difference"], bins = 50)
    plt.title("Error Distribution")
    plt.show()


from sentence_transformers import SentenceTransformer
import logging

def extract_latex_features(latex_problems, use_trees=False, use_sequences=False):
    if use_trees or use_sequences:
        features = []
        for problem in latex_problems:
            try:
                parsed_expr = parse_latex(problem)
                if parsed_expr:
                    if use_trees:
                        features.append(sympy_to_tree(parsed_expr))
                    elif use_sequences:
                        features.append(sympy_to_sequence(parsed_expr))
                else:
                    features.append(None)
            except Exception as e:
                logging.error(f"Error parsing LaTeX: {e}, problem: {problem}")
                features.append(None)
        return features
    else:
        try:
            model = SentenceTransformer('all-mpnet-base-v2')
            embeddings = model.encode(latex_problems)
            return embeddings
        except OSError as e:
            logging.error(f"Error loading SentenceTransformer: {e}")
            return []
        except ValueError as e:
            logging.error(f"Error encoding with SentenceTransformer: {e}")
            return []
        except Exception as e:
            logging.error(f"Unkown error with SentenceTransformer: {e}")
            return []

def extract_math_features(latex_str):
    """Extracts mathematical features from a LaTeX string."""
    features = {}
    try:
        expr = sympy.parsing.latex.parse_latex(latex_str)
        if expr is None:
            return features

        # Complexity Metrics
        features["num_operators"] = len(re.findall(r'[+\-*/^]', latex_str))
        features["num_variables"] = len(expr.free_symbols)
        features["num_constants"] = len(expr.atoms(sympy.Number))
        features["latex_length"] = len(latex_str)
        features["num_trig"] = len(re.findall(r'\\sin|\\cos|\\tan', latex_str))
        features["num_factorials"] = len(re.findall(r'!', latex_str))

        # Symbolic Properties
        features["has_fraction"] = int(any(isinstance(arg, sympy.Rational) for arg in sympy.preorder_traversal(expr)))
        features["has_sqrt"] = int(any(isinstance(arg, sympy.Pow) and arg.args[1] == sympy.Rational(1, 2) for arg in sympy.preorder_traversal(expr)))
        features["has_abs"] = int(len(re.findall(r'\|', latex_str)) > 0)
        features["has_binomial"] = int(len(re.findall(r'\\binom', latex_str)) > 0)

        #Numerical properties
        constants = list(expr.atoms(sympy.Number))
        if constants:
            features['max_constant'] = float(max(constants))
            features['min_constant'] = float(min(constants))
        else:
            features['max_constant'] = 0
            features['min_constant'] = 0
        features['only_integers'] = int(all(isinstance(i, sympy.Integer) for i in constants))

        return features

    except Exception as e:
        logging.error(f"Error extracting features: {e}, LaTeX: {latex_str}")
        return features

# Example Usage
latex_problem = r"\frac{1}{2} + \sqrt{4} + 3! + \binom{5}{2} + |{-5}| + 2x^2 + 3"
features = extract_math_features(latex_problem)
print(features)

def train_supervised_model(train_df, val_df, model_name="t5-base"):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

    def tokenize_data(examples):
        inputs = tokenizer(examples["problem"], truncation=True, padding="max_length", max_length=256)
        labels = tokenizer(examples["answer"], truncation=True, padding="max_length", max_length=64)
        return {"input_ids": inputs.input_ids, "attention_mask": inputs.attention_mask, "labels": labels.input_ids}

    train_dataset = Dataset.from_pandas(train_df)
    train_dataset = train_dataset.map(tokenize_data, batched=True)

    val_dataset = Dataset.from_pandas(val_df)
    val_dataset = val_dataset.map(tokenize_data, batched=True)

    training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=200,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    run_name="my_t5_training_run",
    report_to="tensorboard",
    logging_steps=10,
    evaluation_strategy="epoch", # Evaluate at the end of each epoch
    learning_rate=1e-4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
    save_strategy = "epoch"  # Save at the end of each epoch to match evaluation strategy
)

    # EarlyStopping is now imported and used separately
    from transformers import EarlyStoppingCallback
    early_stopping = EarlyStoppingCallback(early_stopping_patience=5)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[early_stopping], #add it to the callbacks list.
    )

    trainer.train()
    return trainer.model, tokenizer

def predict(latex_problem, model, tokenizer):
    device = model.device
    inputs = tokenizer(latex_problem, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    print(f"Input Tensor Device: {inputs.input_ids.device}")
    print(f"Input Ids : {inputs.input_ids}")
    print(f"Decoded Input: {tokenizer.decode(inputs.input_ids[0])}")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=20)
    print(f"Output Ids: {outputs}") #Print the output Ids
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

ERROR:root:Error extracting features: module 'sympy.parsing' has no attribute 'latex', LaTeX: \frac{1}{2} + \sqrt{4} + 3! + \binom{5}{2} + |{-5}| + 2x^2 + 3


{}


# ***Supervised Model with Unsupervised Feature Extraction***

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")

def main():
    csv_file_path = '/content/drive/MyDrive/AI_Math_Olympiad/reference.csv'
    pdf_file_path = '/content/drive/MyDrive/AI_Math_Olympiad/reference.pdf'

    all_problems = []
    all_answers = []

    try:
        # Load from CSV if it exists
        try:
            csv_df = pd.read_csv(csv_file_path)
            all_problems.extend(csv_df['problem'].tolist())
            all_answers.extend(csv_df['answer'].astype(str).tolist())
        except FileNotFoundError:
            print(f"CSV file not found at {csv_file_path}. Proceeding with PDF.")

        # Load from PDF if it exists
        pdf_problems, pdf_answers = extract_problems_from_pdf(pdf_file_path)
        all_problems.extend(pdf_problems)
        all_answers.extend(pdf_answers)

        if not all_problems:
            print("No data loaded from CSV or PDF. Exiting.")
            return

        df = pd.DataFrame({"problem": all_problems, "answer": all_answers})
        train_df, val_df = train_test_split(df, test_size=0.2)

        supervised_model, supervised_tokenizer = train_supervised_model(train_df, val_df)

        model_save_path = "/content/drive/MyDrive/AI_Math_Olympiad/T5_&_Tokenizer/trained_t5_model"
        tokenizer_save_path = "/content/drive/MyDrive/AI_Math_Olympiad/T5_&_Tokenizer/trained_t5_tokenizer"

        supervised_model.save_pretrained(model_save_path)
        supervised_tokenizer.save_pretrained(tokenizer_save_path)
        print(f"Model saved to: {model_save_path}")
        print(f"Tokenizer saved to: {tokenizer_save_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/AI_Math_Olympiad/reference.pdf'
    extracted_problems, extracted_answers = extract_problems_from_pdf(pdf_file)

    if extracted_problems:
        print(f"Found {len(extracted_problems)} problems in '{pdf_file}':")
        for i, problem in enumerate(extracted_problems):
            print(f"Problem {i+1}: {problem}")
            if i < len(extracted_answers):
                print(f"Answer {i+1}: {extracted_answers[i]}")
            print("-" * 20)
            main()
    else:
        print(f"No problems found in '{pdf_file}'.")


GPU is available and being used.
Found 10 problems in '/content/drive/MyDrive/AI_Math_Olympiad/reference.pdf':
Problem 1: Three airline companies operate flights from Dodola island. Each company has a different schedule of
departures. The first company departs every 100 days, the second every 120 days and the third every
150 days. What is the greatest positive integer dfor which it is true that there will be dconsecutive
days without a flight from Dodola island, regardless of the departure times of the various airlines?
Answer 1: 79
Solution: The airlines are called A100, A120 and A150, labelled by the frequency of their depar-
tures. We first prove that there is a period of 99 days after an A100 departure during which no
A150 plane takes off.
Consider a period of 301 days which starts with an A150 departure on Day 0, followed by a departure
on Day 150 and on Day 300. Let the first A100 departure in this period be on Day x.
There are two possibilities: (i) 0 ≤x≤50 or (ii) 51 ≤x≤99. In 

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,9.160959
2,No log,7.549445
3,No log,6.517236
4,No log,5.386489
5,7.479400,4.263592
6,7.479400,3.195917
7,7.479400,2.555715
8,7.479400,2.276217
9,7.479400,2.202902
10,2.226200,2.187646


...

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,9.160959
2,No log,7.549445
3,No log,6.517236
4,No log,5.386489
5,7.479400,4.263592
6,7.479400,3.195917
7,7.479400,2.555715
8,7.479400,2.276217
9,7.479400,2.202902
10,2.226200,2.187646


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Model saved to: /content/drive/MyDrive/AI_Math_Olympiad/T5_&_Tokenizer/trained_t5_model
Tokenizer saved to: /content/drive/MyDrive/AI_Math_Olympiad/T5_&_Tokenizer/trained_t5_tokenizer


Testing

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 1. Load your trained model and tokenizer (replace with your actual paths)
model_path = "/content/drive/MyDrive/AI_Math_Olympiad/T5_&_Tokenizer/trained_t5_model"
tokenizer_path = "/content/drive/MyDrive/AI_Math_Olympiad/T5_&_Tokenizer/trained_t5_tokenizer"

try:
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    print(f"Model Device: {model.device}")

    # Small data test
    # Load a small dataframe for testing (replace with your actual data loading)
    data = {"problem": [r"2 + 2", r"\frac{10}{2}", r"5 * 3", r"10 - 4", r"sqrt(16)", r"3+4", r"5*6", r"7/1", r"8-2", r"10+1"], "answer": ["4", "5", "15", "6", "4", "7", "30", "7", "6", "11"]}
    train_df = pd.DataFrame(data)
    small_df = train_df.head(10)
    latex_problems = small_df['problem'].tolist()

    for problem in latex_problems:
        prediction = predict(problem, model, tokenizer)
        print(f"Small data test Problem: {problem}, Prediction: {prediction}")

except OSError as e:
    print(f"Error loading model or tokenizer: {e}")
    exit()

except Exception as e:
    print(f"An error occurred during prediction: {e}")

# 2. Define the LaTeX problems
latex_problems = [
    r"2 + 2",
    r"\frac{10}{2}",
    r"5 * 3",
    r"10 - 4",
    r"sqrt(16)"
]

# 3. Call the predict function for each problem and print the predictions
for problem in latex_problems:
    prediction = predict(problem, model, tokenizer)
    print(f"Problem: {problem}, Prediction: {prediction}")

# ***Data Augmentation***

Augmented Variables

In [ ]:
import sympy
import random
import re

def change_variable_names(latex_str):
    """Changes variable names in a LaTeX string."""
    variables = re.findall(r'\$([a-zA-Z])\$', latex_str)
    variables = list(set(variables))
    if not variables:
        return latex_str
    new_variables = random.sample('abcdefghijklmnopqrstuvwxyz', len(variables))
    variable_map = dict(zip(variables, new_variables))

    def replace_variable(match):
        var = match.group(1)
        if var in variable_map:
            return '$' + variable_map[var] + '$'
        return match.group(0)

    return re.sub(r'\$([a-zA-Z])\$', replace_variable, latex_str)

# Example Usage
latex_problem = r"Find the value of $x^2 + 2y$ when $x=3$ and $y=4$."
varied_problem = change_variable_names(latex_problem)
print(f"Original: {latex_problem}")
print(f"Varied: {varied_problem}")

Original: Find the value of $x^2 + 2y$ when $x=3$ and $y=4$.
Varied: Find the value of $x^2 + 2y$ when $x=3$ and $y=4$.


Augmented Problems

In [ ]:
import nltk
from nltk.corpus import wordnet
import random

def rephrase_problem(text):
    """Rephrases a problem by substituting synonyms."""
    words = nltk.word_tokenize(text)
    rephrased_words = []
    for word in words:
        synsets = wordnet.synsets(word)
        if synsets:
            synonyms = [lemma.name() for synset in synsets for lemma in synset.lemmas()]
            if synonyms:
                rephrased_words.append(random.choice(synonyms))
            else:
                rephrased_words.append(word)
        else:
            rephrased_words.append(word)
    return ' '.join(rephrased_words)

# Example Usage
nltk.download('punkt')
nltk.download('wordnet')
problem_text = "Calculate the sum of 5 and 7."
rephrased_text = rephrase_problem(problem_text)
print(f"Original: {problem_text}")
print(f"Rephrased: {rephrased_text}")

Augmented Expressions

In [ ]:
import sympy
import random

def transform_expression(latex_str):
    """Applies random mathematical transformations."""
    expr = sympy.parsing.latex.parse_latex(latex_str)
    if not expr:
        return latex_str

    if random.random() < 0.3:
        # Change constant
        constant = random.randint(-5, 5)
        expr = expr + constant
    elif random.random() < 0.3:
        # Rearrange terms
        terms = list(expr.args)
        random.shuffle(terms)
        expr = sum(terms)

    return sympy.latex(expr)

# Example Usage
latex_expr = r"x^2 + 2x + 1"
transformed_expr = transform_expression(latex_expr)
print(f"Original: {latex_expr}")
print(f"Transformed: {transformed_expr}")

Augmented Context

In [ ]:
def vary_context(text):
    """Adds or removes contextual information."""
    if random.random() < 0.5:
        return text + " Assume all units are in meters."
    else:
        return text.replace("Assume all units are in meters.", "")

Create Synthetic Problems & Solutions

In [ ]:
import sympy
import random

def generate_synthetic_problem():
    """Generates a random synthetic problem."""
    operators = ['+', '-', '*', '/', '**']
    functions = ['sympy.sqrt', 'sympy.sin', 'sympy.cos', 'sympy.factorial']
    symbols = ['x', 'y', 'z']

    def generate_term(depth):
        if depth > 3:
            if random.random() < 0.5:
                return str(random.randint(-10, 10))
            else:
                return random.choice(symbols)
        else:
            if random.random() < 0.3:
                return str(random.randint(-10, 10))
            elif random.random() < 0.3:
                return random.choice(symbols)
            elif random.random() < 0.4:
                operator = random.choice(operators)
                left = generate_term(depth + 1)
                right = generate_term(depth + 1)
                return f"({left} {operator} {right})"
            else:
                func = random.choice(functions)
                arg = generate_term(depth + 1)
                return f"{func}({arg})"

    expression_str = generate_term(0)
    return expression_str

def generate_synthetic_solution(expression_str):
    """Generates the solution for a synthetic problem."""
    try:
        expr = sympy.parsing.sympy_parser.parse_expr(expression_str)
        solution = expr.evalf()
        solution_modulo_1000 = int(round(solution)) % 1000
        return solution_modulo_1000
    except Exception as e:
        print(f"Error evaluating expression: {e}")
        return None

# Generate and test some examples:
for _ in range(5):
    problem = generate_synthetic_problem()
    solution = generate_synthetic_solution(problem)
    if solution is not None:
        print(f"Problem: {problem}, Solution: {solution}")

Error evaluating expression: Cannot round symbolic expression
Problem: -6, Solution: 994
Error evaluating expression: 'Symbol' object has no attribute 'sqrt'
Problem: 6, Solution: 6
Error evaluating expression: 'Symbol' object has no attribute 'factorial'


# ***Submissions***

In [ ]:
import requests
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# API Configuration (Replace with your actual API details)
API_ENDPOINT = "YOUR_API_ENDPOINT"
API_KEY = "YOUR_API_KEY"

def submit_solution(problem_id, solution):
    """Submits a solution to the AIMO API."""
    try:
        # Format the solution according to the API's requirements
        payload = {
            "problem_id": problem_id,
            "solution": solution,
            "api_key": API_KEY,
        }
        # Send the solution to the API endpoint
        response = requests.post(API_ENDPOINT, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        # Parse the API response
        result = response.json()
        logging.info(f"Solution submitted successfully: {result}")
        return result

    except requests.exceptions.RequestException as e:
        logging.error(f"API request error: {e}")
        return None
    except json.JSONDecodeError as e:
        logging.error(f"API response decoding error: {e}, Response: {response.text}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occured: {e}")
        return None

# Example Usage (Integration with your prediction function)
def solve_and_submit(problem_data, model, tokenizer):
    """Solves a problem and submits the solution."""
    try:
        problem_id = problem_data["id"]
        latex_problem = problem_data["latex"]
        prediction = predict(latex_problem, model, tokenizer) #Your prediction function.
        submit_solution(problem_id, prediction)
    except Exception as e:
        logging.error(f"Error during problem solving and submission: {e}")

# Example Integration (with a test problem)
test_problem = {"id": "123", "latex": r"$(2+3)^2$"}
#solve_and_submit(test_problem, your_model, your_tokenizer)